In [ ]:
!pip install tensorflow
!pip install tensorflow_datasets
!pip install tensorflow_hub
!pip install keras_cv
!pip install keras-cv tensorflow --upgrade
# apt-get update && apt-get install -y python3-opencv
!pip install opencv-python
!pip install opencv-python opencv-python-headless
!pip install pycocotools

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow.keras.layers as layers
import tensorflow_hub as hub

import baggage_dataset_single_labels

%load_ext tensorboard
%load_ext autoreload
%autoreload 2

In [ ]:
print('jfdkal')

In [11]:
def resize_and_normalize_img(example):
  """Normalizes images: `uint8` -> `float32`."""
  example['image'] = tf.image.resize(example['image'], [256, 256])
  example['image'] = tf.cast(example['image'], tf.float32) / 255.
  return example['image'], example['label']

def reduce_for_classification(example):
        #for key in ['image/filename', 'labels_no_difficult', 'objects']:
        #    example.pop(key)
        return example

model_handle = "https://tfhub.dev/tensorflow/ssd_mobilenet_v2/fpnlite_320x320/1"    

(train_ds, test_ds), ds_info = tfds.load('MyDatasetBuilder', split=['train[:20%]', 'train[20%:]'], with_info=True)

train_ds_class = train_ds.map(reduce_for_classification, num_parallel_calls=tf.data.AUTOTUNE)
train_ds_class = train_ds_class.map(resize_and_normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
train_ds_class = train_ds_class.map(lambda x, y: (x, tf.reduce_sum(tf.one_hot(y, 20, axis=-1), axis=0)))

train_ds_class = train_ds_class.cache()
train_ds_class = train_ds_class.shuffle(ds_info.splits['train'].num_examples)
train_ds_class = train_ds_class.batch(64)
train_ds_class = train_ds_class.prefetch(tf.data.AUTOTUNE)

inputs = layers.Input(shape=[256, 256, 3], name='image')
# x = hub.KerasLayer(model_handle, trainable=False,)
# bbox = x.layers['detection_boxes']
x = tf.keras.layers.Flatten()(inputs)
bbox = layers.Dense(4, activation='sigmoid')(x)
clabel = layers.Dense(6, activation='softmax')(x)

# outputs = layers.Concatenate(clabel, bbox)

model = tf.keras.Model(inputs, [bbox, clabel])
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')

model.fit(train_ds_class, epochs=5)

TypeError: Failed to construct dataset my_dataset_builder: 'baggage_dataset_single_labels' is not a package